In [1]:
from docplex.mp.model import Model
from docplex.util.environment import get_environment
from collections import namedtuple
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
pathel=r'E:\Master\Building\Zernez_El.csv'
pathsh=r'E:\Master\Building\Zernez_SH.csv'
pathr=r'E:\docplex-examples-master\RoofPV.xlsx'
pathtemp=r'E:\docplex-examples-master\Temp.xlsx'
pathgshp=r'E:\docplex-examples-master\GSHP.xls'
PV=pd.read_excel(pathr,'PV_full',parse_cols='A:KV') #specific to zernez
ST=pd.read_excel(pathr,'ST_full',parse_cols='A:KV') #specific to zernez
RoofArea=pd.read_excel(pathr,'Area',parse_cols='A:B') #specific to zernez
#HeatDemand=pd.read_csv(pathsh) #specific to zernez
#ElecDemand=pd.read_csv(pathel) #specific to zernez
GSHP=pd.read_excel(pathgshp,'GSHP_all',parse_cols='A:B') #specific to zernez
Temp=pd.read_excel(pathtemp,'Temp',parse_cols='A:B') #specific to zernez
BDemand=pd.DataFrame(index=range(0,8760),columns=['Elec','Heat'])
RenewPotential=pd.DataFrame(index=range(0,8760),columns=['PV','ST'])
HPTemp=Temp.as_matrix()

In [ ]:
bnumber=HeatDemand.columns
bnumber

In [ ]:
def load_data(nbld,HeatDemand,ElecDemand,PV,ST,RoofArea,BDemand,RenewPotential):
    BDemand.Heat=HeatDemand[nbld]
    BDemand.Elec=ElecDemand[nbld]
    RenewPotential.PV=PV[int(nbld)]
    RenewPotential.ST=ST[int(nbld)]
    MaxEvap=GSHP.loc[int(nbld),'Gpotential']
    MaxRoof=RoofArea.loc[int(nbld),'Area']
    Demand=BDemand.as_matrix()
    RPotential=RenewPotential.as_matrix()
    return [Demand,RPotential,MaxRoof,MaxEvap]

In [ ]:
HPTech = [
    ("GSHP",8889.2,2178,0.11,2.8,15.0,5.5,0,1,1200),
    ("ASHP",11013,1332.5,0.11,2.8,15.0,5.5,0,1,500)]
BTech =[
    ("Gas-boiler",12658,605.7,0.9, 25.0,3.7,0,3,1,10000),
    ("Oil-boiler",17267,463.63,0.85,25.0,3.7,0,5,1,10000),
    ("Biomass-boiler",27404,1024.2, 0.75, 20.0,0,0.0054,4,1,1000)]
CHPTech = [("PEMFC",53158,5748.2,0.5848,0.0004,0.15,1.9,11.5,0.025,2,0,100,0.5852),
    ("PEMEC",23729,1975.5,0.5485,-0.0006,0.15,1.0894,9,0.025,0,2,100,0.5452)]
RTech = [("PV",1075.7,267.15,0,0.034,30,6,0),
    ("ST",5608.5,1671.5,9,0,30,6,1)]
CompTech=[("H2Comp",300,0.04,15,2,0)]
StorageTech=[("Battery",2000,943.15,10,25,0.92,0.92,0.001,0.5,0.5,0,1000),
    ("TES",1685,12.604,0,17,0.9,1,0.01,0.25,0.25,1,1000),
    ("Hydrogen",620,566,0,20,1,1,0,1,1,2,4000)]#prices in KG!

FuelProperties=[("Elec",0.198,0.029),
                ("Heat",0,0),
                ("Hydrogen",0,0),
                ("NG",0.067,0.198),
                ("Biomass",0.075,0),
                ("Oil",0.062,0.270),
                ("Solar",0,0),
                ("Water",0,0)]
                
#CHPtech=[    ("MGT", 900, 0.3, 10.0,0,0.01)]
#0=Elec, 1=Heat, 2=Hydrogen, 3=Natural gas,4=biomass,5=oil
discount=0.06  
Horizon=range(0,8760)
hps=range(0,len(HPTech))
boil=range(0,len(BTech))
renew=range(0,len(RTech))
chp=range(0,len(CHPTech))
stor=range(0,len(StorageTech))
comp=range(0,len(CompTech))
DemandCarrier=range(0,2)
fuel=range(0,len(FuelProperties))
techr=namedtuple("RTech",["name","fixed_cost","var_cost","OMF","OMV","Lifetime","ECin","ECout"])
techhp = namedtuple("HPTech", ["name", "fixed_cost","var_cost", "eff_slope","eff_int", "Lifetime","OMF","ECin","ECout","MaxCap"])
techb = namedtuple("BTech", ["name", "fixed_cost","var_cost", "eff", "Lifetime","OMF","OMV","ECin","ECout","MaxCap"])
techchp = namedtuple("CHPTech",["name","fixed_cost","var_cost","eff_slope","eff_int","Lower","Upper","Lifetime","OMV","ECin","ECout","MaxCap","OutFactor"])
carrier= namedtuple("FuelProperties",["name","FuelCost","FuelCO2"])
techcomp = namedtuple("CompTech",["name","unit_cost","eff","Lifetime","ECin","ECout"])
strge=namedtuple("StorageTech",["name","fixed_cost","var_cost","OMF","Lifetime","ChrgEff","DschEff","Decay","MaxChrg","MaxDsch","Carrier","MaxCap"])
TechHP = [techhp(*p) for p in HPTech]
TechB = [techb(*b) for b in BTech]
TechRenew = [techr(*r) for r in RTech]
TechCHP = [techchp(*d) for d in CHPTech]
FuelProp = [carrier(*f) for f in FuelProperties]
TechStor = [strge(*s) for s in StorageTech]
CompTech = [techcomp(*c) for  c in CompTech]
AnnuityHP=[None] * len(HPTech)
AnnuityB=[None] * len(BTech)
AnnuityCHP=[None] * len(CHPTech)
AnnuityR=[None]*len(RTech)
AnnuityS=[None]*len(StorageTech)
AnnuityComp=[None]*len(CompTech)
COP=pd.DataFrame(index=Horizon,columns=['GSHP','ASHP'])
for h in Horizon:
    COP.loc[h,'GSHP']=Temp.loc[h,'GroundTemp']*TechHP[0].eff_slope+TechHP[0].eff_int
    COP.loc[h,'ASHP']=Temp.loc[h,'AirTemp']*TechHP[1].eff_slope+TechHP[1].eff_int
for p in hps:
    AnnuityHP[p]=discount/(1-(1/((1+discount)**TechHP[p].Lifetime)))
for b in boil:
    AnnuityB[b]=discount/(1-(1/((1+discount)**TechB[b].Lifetime)))
for d in chp:
    AnnuityCHP[d]=discount/(1-(1/((1+discount)**TechCHP[d].Lifetime)))
for r in renew:
    AnnuityR[r]=discount/(1-(1/((1+discount)**TechRenew[r].Lifetime)))
for s in stor:
    AnnuityS[s]=discount/(1-(1/((1+discount)**TechStor[s].Lifetime)))
for c in comp:
    AnnuityComp[c]=discount/(1-(1/((1+discount)**CompTech[c].Lifetime)))
COPHP=COP.as_matrix()

In [8]:
def build_model(MaxEvap,MaxRoof,Demand,RPotential,hps,boil,renew,chp,TechHP,TechB,TechRenew,TechCHP,FuelProp,TechStor,CompTech,AnnuityHP,AnnuityB,AnnuityCHP,AnnuityR,AnnuityS,AnnuityComp,COPHP):
    mdl = Model(name='BuildingHub')
    Horizon = range(0, len(Demand))
    # Sizing decision variables
    mdl.SizeHP = mdl.integer_var_list(hps,lb=0,name='SizeHP')
    mdl.SizeBoiler = mdl.integer_var_list(boil,lb=0,name='SizeBoiler')
    mdl.SizeCHP = mdl.integer_var_list(chp,lb=0,name='SizeCHP')
    mdl.SizeR = mdl.integer_var_list(renew,lb=0,name='SizeR')
    mdl.SizeStor = mdl.integer_var_list(stor,lb=0,name='SizeStor')
    mdl.SizeComp = mdl.integer_var(lb=0,name='SizeComp')

    #Installation Binaries
    mdl.InstBoiler=mdl.binary_var_list(boil,name='InstBoiler')
    mdl.InstCHP=mdl.binary_var_list(chp,name='InstCHP')
    mdl.InstHP=mdl.binary_var_list(hps,name='InstHP')
    mdl.InstR=mdl.binary_var_list(renew,name='InstR')
    mdl.InstStor=mdl.binary_var_list(stor,name='InstStor')
    mdl.add_constraints(mdl.SizeBoiler[b]<=TechB[b].MaxCap*mdl.InstBoiler[b] for b in boil)
    mdl.add_constraints(mdl.SizeCHP[d]<=TechCHP[d].MaxCap*mdl.InstCHP[d] for d in chp)
    mdl.add_constraints(mdl.SizeHP[p]<=TechHP[p].MaxCap*mdl.InstHP[p] for p in hps)
    mdl.add_constraints(mdl.SizeStor[s]<=TechStor[s].MaxCap*mdl.InstStor[s] for s in stor)
    mdl.add_constraint(mdl.SizeR[0]<=MaxRoof*mdl.InstR[0])
    mdl.add_constraint(mdl.SizeR[1]<=MaxRoof*mdl.InstR[1])
    #mdl.add_constraint(mdl.SizeR[2]==2300)

    mdl.add_constraint(mdl.sum(mdl.InstBoiler[b] for b in boil)==1)
    mdl.add_constraint(mdl.sum(mdl.InstHP[p] for p in hps)==1)
    #Aux variable

    #mdl.yBoiler=mdl.binary_var_matrix(boil,Horizon,name='yBoiler')
    mdl.yCHP=mdl.binary_var_matrix(chp,Horizon,name='yCHP')
    #mdl.sdCHP=mdl.binary_var_matrix(chp,Horizon,name='sdCHP')
    #mdl.suCHP=mdl.binary_var_matrix(chp,Horizon,name='suCHP')
    #mdl.DHPump=mdl.continuous_var_list(Horizon,lb=0,name='DHPump')
    mdl.AuxCHP=mdl.continuous_var_matrix(chp,Horizon,lb=0,name='AuxCHP')
    #mdl.AuxCHPsd=mdl.continuous_var_matrix(chp,Horizon,lb=0,name='AuxCHPsd')
    #mdl.AuxCHPsu=mdl.continuous_var_matrix(chp,Horizon,lb=0,name='AuxCHPsu')
    #Aux constraints

    mdl.add_constraints(mdl.AuxCHP[d,h]<=mdl.yCHP[d,h]*TechCHP[d].MaxCap for d in chp for h in Horizon)
    mdl.add_constraints(mdl.AuxCHP[d,h]<=mdl.SizeCHP[d] for d in chp for h in Horizon)
    mdl.add_constraints(mdl.AuxCHP[d,h]>=mdl.SizeCHP[d]-TechCHP[d].MaxCap*(1-mdl.yCHP[d,h]) for d in chp for h in Horizon)

    #mdl.add_constraints(mdl.AuxCHPsd[d,h]<=mdl.sdCHP[d,h]*TechCHP[d].MaxCap for h in Horizon for d in chp)
    #mdl.add_constraints(mdl.AuxCHPsd[d,h]<=mdl.SizeCHP[d] for h in Horizon for d in chp)
    #mdl.add_constraints(mdl.AuxCHPsd[d,h]>=mdl.SizeCHP[d]-TechCHP[d].MaxCap*(1-mdl.sdCHP[d,h]) for h in Horizon for d in chp)
    #mdl.add_constraints(mdl.AuxCHPsu[d,h]<=mdl.suCHP[d,h]*TechCHP[d].MaxCap for h in Horizon)
    #mdl.add_constraints(mdl.AuxCHPsu[d,h]<=mdl.SizeCHP[d] for h in Horizon for d in chp)
    #mdl.add_constraints(mdl.AuxCHPsu[d,h]>=mdl.SizeCHP[d]-TechCHP[d].MaxCap*(1-mdl.suCHP[d,h]) for h in Horizon)
    #Operation decision variables
    mdl.HPOut = mdl.continuous_var_matrix(hps,Horizon,lb=0,name='HPOut')
    mdl.HPIn = mdl.continuous_var_matrix(hps,Horizon,lb=0,name='HPIn')
    mdl.BoilOut=mdl.continuous_var_matrix(boil,Horizon,lb=0,name='BoilOut')
    mdl.BoilIn=mdl.continuous_var_matrix(boil,Horizon,lb=0,name='BoilIn')
    mdl.CompElec = mdl.continuous_var_list(Horizon,lb=0,name='CompElec')
    mdl.CHPIn = mdl.continuous_var_matrix(chp,Horizon,lb=0,name='CHPIn')
    mdl.CHPOut = mdl.continuous_var_matrix(chp,Horizon,lb=0,name='CHPOut')
    mdl.CHPheat=mdl.continuous_var_list(Horizon,lb=0,name='CHPheat')
    mdl.ROut = mdl.continuous_var_matrix(renew,Horizon,lb=0,name='ROut')
    mdl.ElecIn = mdl.continuous_var_list(Horizon,lb=0,name='ElecIn')
    mdl.ElecOut = mdl.continuous_var_list(Horizon,lb=0,name='ElecOut')
    mdl.HeatDump = mdl.continuous_var_list(Horizon,lb=0,name='HeatDump')
    mdl.NGIn = mdl.continuous_var_list(Horizon,lb=0,name='NGIn')
    mdl.StorCharge = mdl.continuous_var_matrix(stor,Horizon,lb=0,name='StorCharge')
    mdl.StorDischarge = mdl.continuous_var_matrix(stor,Horizon,lb=0,name='StorDischarge')
    mdl.StorLevel = mdl.continuous_var_matrix(stor,Horizon,lb=0,name='StorLevel')
    #mdl.DI = mdl.continuous_var_list(Horizon,lb=0,name='DI')
    #mdl.add_constraints(mdl.DHPump[h]==0.04*Demand[h,1] for h in Horizon)
    ##COST DECISION VARIABLES
    #Capital costs

    #capcost
    mdl.HPCapCost = mdl.continuous_var_list(hps,lb=0,name='HPCapCost')
    mdl.BoilCapCost = mdl.continuous_var_list(boil,lb=0,name='BoilCapCost')
    mdl.CompCapCost = mdl.continuous_var(lb=0,name='CompCapCost')
    mdl.StorCapCost = mdl.continuous_var_list(stor,lb=0,name='StorCapCost')
    mdl.CHPCapCost = mdl.continuous_var_list(chp,lb=0,name='CHPCapCost')
    mdl.RCapCost = mdl.continuous_var_list(renew,lb=0,name='RCapCost')
    #O&M cost
    mdl.OMFCostHP = mdl.continuous_var_list(hps,lb=0,name='OMFCostHP')
    mdl.OMFCostBoil = mdl.continuous_var_list(boil,lb=0,name='OMFCostBoil')
    mdl.OMFCostStor = mdl.continuous_var_list(stor,lb=0,name='OMFCostStor')
    mdl.OMVCostBoil = mdl.continuous_var_list(boil,lb=0,name='OMVCostBoil')
    mdl.OMVCostCHP = mdl.continuous_var_list(chp,lb=0,name='OMVCostCHP')
    mdl.OMFCostR = mdl.continuous_var_list(renew,lb=0,name='OMFCostR')
    mdl.OMVCostR = mdl.continuous_var_list(renew,lb=0,name='OMVCostR')
    #Fuel costs
    mdl.NGCost = mdl.continuous_var(lb=0,name='NGCost')
    mdl.OilCost = mdl.continuous_var(lb=0,name='OilCost')
    mdl.WoodCost = mdl.continuous_var(lb=0,name='WoodCost')
    mdl.ElecCost = mdl.continuous_var(lb=0,name='ElecCost')
    mdl.FITProfit = mdl.continuous_var(lb=0,name='FITProfit')
    #Cost objective
    mdl.TotalCost = mdl.continuous_var(lb=0,name='TotalCost')
    #CO2 decision variables
    mdl.ElecCO2 = mdl.continuous_var(lb=0,name='ElecCO2')
    mdl.NGCO2 = mdl.continuous_var(lb=0,name='NGCO2')
    mdl.OilCO2 = mdl.continuous_var(lb=0,name='OilCO2')
    mdl.TotalCO2 = mdl.continuous_var(lb=0,name='TotalCO2')

    #MaxCapConstraint
    mdl.add_constraints(TechHP[p].MaxCap>=mdl.SizeHP[p] for p in hps)
    mdl.add_constraints(TechCHP[d].MaxCap>=mdl.SizeCHP[d] for d in chp)
    mdl.add_constraints(TechB[b].MaxCap>=mdl.SizeBoiler[b] for b in boil)
    #Sizing constraint
    mdl.add_constraints(mdl.SizeHP[p] >=mdl.HPOut[p,h] for p in hps for h in Horizon)
    mdl.add_constraints(mdl.SizeBoiler[b]>=mdl.BoilOut[b,h] for b in boil for h in Horizon)
    mdl.add_constraints(mdl.AuxCHP[d,h]>=mdl.CHPIn[d,h]*TechCHP[d].Upper for d in chp for h in Horizon)
    mdl.add_constraints(mdl.AuxCHP[d,h]*TechCHP[d].Lower<=mdl.CHPIn[d,h] for d in chp for h in Horizon)
    mdl.add_constraints(mdl.SizeStor[s]>=mdl.StorLevel[s,h] for s in stor for h in Horizon)
    mdl.add_constraints(mdl.SizeComp>=mdl.CompElec[h] for h in Horizon)
    #Simple eff conv
    mdl.add_constraints(mdl.HPOut[p,h]==mdl.HPIn[p,h]*COPHP[h,p] for p in hps for h in Horizon)
    mdl.add_constraints(mdl.ROut[0,h]==RPotential[h,0]*mdl.SizeR[0] for h in Horizon) #
    mdl.add_constraints(mdl.ROut[1,h]==RPotential[h,1]*mdl.SizeR[1] for h in Horizon) #
    #mdl.add_constraints(mdl.ROut[2,h]==RPotential[h,2]*mdl.SizeR[2]/2300 for h in Horizon)
    mdl.add_constraint(mdl.sum(mdl.SizeR[r] for r in renew if TechRenew[r].ECin==6)<=MaxRoof)

    #CHP eff
    mdl.add_constraints(mdl.CHPOut[d,h]==mdl.CHPIn[d,h]*TechCHP[d].eff_slope+mdl.AuxCHP[d,h]*TechCHP[d].eff_int for d in chp for h in Horizon)
    mdl.add_constraints(mdl.CHPOut[d,h]<=mdl.AuxCHP[d,h]*TechCHP[d].OutFactor for d in chp for h in Horizon)
    mdl.add_constraints(mdl.CHPOut[0,h]*(1.71-1)==mdl.CHPheat[h] for h in Horizon)
    #Boiler eff
    mdl.add_constraints(mdl.BoilOut[b,h]==mdl.BoilIn[b,h]*TechB[b].eff for b in boil for h in Horizon)
    #Comp elec
    mdl.add_constraints(mdl.CompElec[h]==mdl.StorCharge[2,h]*CompTech[p].eff for p in comp for h in Horizon)
    #HP constraint
    mdl.add_constraints(mdl.HPOut[0,h]-mdl.HPIn[0,h]<=MaxEvap for h in Horizon)
    #Energy balances
    mdl.add_constraints(mdl.sum(mdl.HPOut[p,h] for p in hps)+mdl.sum(mdl.BoilOut[b,h] for b in boil)+mdl.CHPheat[h]+mdl.sum(mdl.ROut[r,h] for r in renew if TechRenew[r].ECout==1)+mdl.sum(mdl.StorDischarge[s,h] for s in stor if TechStor[s].Carrier==1)==Demand[h,1]+mdl.sum(mdl.StorCharge[s,h] for s in stor if TechStor[s].Carrier==1)+mdl.HeatDump[h] for h in Horizon)
    mdl.add_constraints(mdl.CHPOut[0,h]+mdl.sum(mdl.ROut[r,h] for r in renew if TechRenew[r].ECout==0)+mdl.sum(mdl.StorDischarge[0,h])+mdl.ElecIn[h]==Demand[h,0]+mdl.sum(mdl.HPIn[p,h] for p in hps)+mdl.ElecOut[h]+mdl.CompElec[h]+mdl.sum(mdl.StorCharge[0,h])+mdl.sum(mdl.CHPIn[1,h]) for h in Horizon)
    mdl.add_constraints(mdl.BoilIn[0,h]==mdl.NGIn[h] for h in Horizon) #assuming no MGT
    mdl.add_constraints(mdl.CHPIn[0,h]==mdl.StorDischarge[2,h] for h in Horizon) #assuming no direct injection
    mdl.add_constraints(mdl.CHPOut[1,h]==mdl.StorCharge[2,h] for h in Horizon)
    #mdl.add_constraints(mdl.DI[h]<=0.03606*mdl.BoilIn[0,h] for h in Horizon)
    #Storage Constraints
    for h in Horizon:
        if h==0:
            mdl.add_constraints(mdl.StorLevel[s,h]==mdl.StorLevel[s,8759] for s in stor)
            mdl.add_constraints(mdl.StorDischarge[s,h]==0 for s in stor)
            mdl.add_constraints(mdl.StorCharge[s,h]==0 for s in stor)
        else:
            mdl.add_constraints(mdl.StorLevel[s,h]==mdl.StorLevel[s,h-1]*(1-TechStor[s].Decay)+mdl.StorCharge[s,h]*TechStor[s].ChrgEff-mdl.StorDischarge[s,h]*(1/TechStor[s].DschEff) for s in stor)
    mdl.add_constraints(mdl.StorCharge[s,h]<=TechStor[s].MaxChrg*mdl.SizeStor[s] for s in stor for h in Horizon)
    mdl.add_constraints(mdl.StorDischarge[s,h]<=TechStor[s].MaxDsch*mdl.SizeStor[s] for s in stor for h in Horizon)
    #FIT Constraints
    mdl.add_constraints(mdl.ROut[0,h]>=mdl.ElecOut[h] for h in Horizon)
    #mdl.add_constraints((1-y_buysell[h])*10000>=ElecIn[h] for h in Horizon)
    ##ramp constraints
    #for h in Horizon:
     #   if h==0:
            #mdl.add_constraints(mdl.sdCHP[d,h]==0 for d in chp)
            #mdl.add_constraints(mdl.suCHP[d,h]==0 for d in chp)
        #else:
            #mdl.add_constraints(mdl.suCHP[d,h]-mdl.sdCHP[d,h]==mdl.yCHP[d,h]-mdl.yCHP[d,h-1] for d in chp)
    #for h in Horizon:
        #if h==(len(Demand)-1):
            #mdl.add_constraints(mdl.CHPIn[d,h]>=(0.18*(mdl.AuxCHP[d,h]-mdl.AuxCHPsu[d,h]))+(0.25*0.18*mdl.AuxCHPsu[d,h]) for d in chp)
            #mdl.add_constraints(mdl.CHPIn[d,h]<=(mdl.AuxCHP[d,h]-mdl.AuxCHPsu[d,h])+((1-0.18)*mdl.AuxCHPsu[d,h]*(1-0.25)+0.18*mdl.AuxCHPsu[d,h]) for d in chp)
            #mdl.add_constraints(mdl.AuxCHPsd[d,h]==0 for d in chp)
        #else:
            #mdl.add_constraints(mdl.CHPIn[d,h]>=(0.18*(mdl.AuxCHP[d,h]-mdl.AuxCHPsd[d,h+1]-mdl.AuxCHPsu[d,h]))+(0.25*0.18*mdl.AuxCHPsu[d,h]) for d in chp)#ramp up min
            #mdl.add_constraints(mdl.CHPIn[d,h]<=mdl.AuxCHP[d,h]-mdl.AuxCHPsu[d,h]+((1-0.18)*mdl.AuxCHPsu[d,h]*(1-0.25)+0.18*mdl.AuxCHPsu[d,h]) for d in chp)#ramp up max
            #mdl.add_constraints(mdl.CHPIn[d,h]<=(mdl.AuxCHP[d,h]-mdl.AuxCHPsd[d,h+1])+((1-0.18)*mdl.AuxCHPsd[d,h+1]*(1-0.25)+0.18*mdl.AuxCHPsd[d,h+1])for d in chp)
            #mdl.add_constraints(mdl.AuxCHP[d,h]-mdl.AuxCHPsd[d,h+1]-mdl.AuxCHPsu[d,h]>=0 for d in chp)
    #calculate costs

    #mdl.add_constraint(mdl.CHPIn[1,4000]>=100)
    #mdl.add_constraint(mdl.SizeCHP[1]>=100)#del
    #mdl.add_constraint(mdl.SizeCHP[0]>=100)#del
    #mdl.add_constraint(mdl.SizeStor[0]==0)#del
    #mdl.add_constraint(mdl.SizeStor[1]==0)#del
    #mdl.add_constraint(mdl.SizeStor[2]>=10000)
    mdl.add_constraints(mdl.HPCapCost[p]==AnnuityHP[p]*(TechHP[p].fixed_cost*mdl.InstHP[p]+TechHP[p].var_cost*mdl.SizeHP[p]) for p in hps)
    mdl.add_constraints(mdl.BoilCapCost[b]==AnnuityB[b]*(TechB[b].fixed_cost*mdl.InstBoiler[b]+TechB[b].var_cost*mdl.SizeBoiler[b]) for b in boil)
    mdl.add_constraints(mdl.StorCapCost[s]==AnnuityS[s]*(TechStor[s].fixed_cost*mdl.InstStor[s]+mdl.SizeStor[s]*TechStor[s].var_cost) for s in stor)
    mdl.add_constraints(mdl.CompCapCost==AnnuityComp[p]*CompTech[p].unit_cost*mdl.SizeComp for p in comp)
    mdl.add_constraints(mdl.CHPCapCost[d]==AnnuityCHP[d]*(TechCHP[d].fixed_cost*mdl.InstCHP[d]+TechCHP[d].var_cost*mdl.SizeCHP[d]) for d in chp)
    mdl.add_constraints(mdl.RCapCost[r]==AnnuityR[r]*(TechRenew[r].fixed_cost*mdl.InstR[r]+mdl.SizeR[r]*TechRenew[r].var_cost) for r in renew)
    mdl.add_constraints(mdl.OMFCostHP[p]==TechHP[p].OMF*mdl.SizeHP[p] for p in hps)

    mdl.add_constraints(mdl.OMFCostStor[s]==TechStor[s].OMF*mdl.SizeStor[s] for s in stor)
    mdl.add_constraints(mdl.OMFCostBoil[b]==TechB[b].OMF*mdl.SizeBoiler[b] for b in boil)
    mdl.add_constraints(mdl.OMVCostBoil[b]==mdl.sum(mdl.BoilOut[b,h] for h in Horizon)*TechB[b].OMV for b in boil)
    mdl.add_constraints(mdl.OMVCostCHP[d]==mdl.sum(mdl.CHPOut[d,h]for h in Horizon)*TechCHP[d].OMV for d in chp)
    mdl.add_constraints(mdl.OMFCostR[r]==mdl.SizeR[r]*TechRenew[r].OMF for r in renew)
    mdl.add_constraints(mdl.OMVCostR[r]==mdl.sum(mdl.ROut[r,h] for h in Horizon)*TechRenew[r].OMV for r in renew)
    mdl.add_constraint(mdl.NGCost==0.067*mdl.sum(mdl.NGIn[h] for h in Horizon))
    mdl.add_constraint(mdl.NGCO2==0.20*mdl.sum(mdl.NGIn[h] for h in Horizon))
    mdl.add_constraint(mdl.OilCost==0.062*mdl.sum(mdl.BoilOut[1,h] for h in Horizon))
    mdl.add_constraint(mdl.WoodCost==0.075*mdl.sum(mdl.BoilOut[2,h] for h in Horizon))
    mdl.add_constraint(mdl.OilCO2==0.27*mdl.sum(mdl.BoilOut[1,h] for h in Horizon))
    mdl.add_constraint(mdl.ElecCost==0.198*mdl.sum(mdl.ElecIn[h] for h in Horizon))
    mdl.add_constraint(mdl.FITProfit==0.12*mdl.sum(mdl.ElecOut[h] for h in Horizon)) #assuming to non-renewable eletricity production
    mdl.add_constraint(mdl.ElecCO2==mdl.sum(mdl.ElecIn[h] for h in Horizon)*0.09)
    mdl.add_constraint(mdl.TotalCost==mdl.sum(mdl.HPCapCost[p] for p in hps)+mdl.sum(mdl.BoilCapCost[b] for b in boil)+mdl.WoodCost+mdl.sum(mdl.CHPCapCost[d] for d in chp)+mdl.sum(mdl.RCapCost[r] for r in renew)+mdl.CompCapCost+mdl.sum(mdl.StorCapCost[s] for s in stor)+mdl.sum(mdl.OMFCostHP[p] for p in hps)+mdl.sum(mdl.OMFCostBoil[b] for b in boil)+mdl.sum(mdl.OMFCostR[r] for r in renew)+mdl.sum(mdl.OMFCostStor[s] for s in stor)+mdl.sum(mdl.OMVCostCHP[d] for d in chp)+mdl.sum(mdl.OMVCostR[r] for r in renew)+mdl.sum(mdl.OMVCostBoil[b] for b in boil)-mdl.FITProfit+mdl.NGCost+mdl.ElecCost+mdl.OilCost)
    mdl.add_constraint(mdl.TotalCO2==mdl.ElecCO2+mdl.NGCO2+mdl.OilCO2)
    #mdl.add_constraint(mdl.TotalCO2<=EpConst)
    mdl.add_constraint(mdl.SizeBoiler[2]==0)
    mdl.print_information()
    # Minimize cost
    return mdl

In [10]:
def min_cost(mdl):
    mdl.minimize(mdl.TotalCost)
    return mdl

In [11]:
def min_co2(mdl):
    mdl.minimize(mdl.TotalCost*0.00001+mdl.TotalCO2)
    return mdl

In [16]:
# only building level!
tname=['25','50','75']
#for t in range(1,4):
    #tcase=tname[t-1]
for b in range(0,13):
        #Epconst=Epconstraint.loc[b,str(t)]
    Costname='Cost_'+str(b)
    CO2name='CO2_'+str(b)
    Demand,RPotential,MaxRoof,MaxEvap=load_data(bnumber[b],HeatDemand,ElecDemand,PV,ST,RoofArea,BDemand,RenewPotential)
    mm=build_model(MaxEvap,MaxRoof,Demand,RPotential,hps,boil,renew,chp,TechHP,TechB,TechRenew,TechCHP,FuelProp,TechStor,CompTech,AnnuityHP,AnnuityB,AnnuityCHP,AnnuityR,AnnuityS,AnnuityComp,COPHP)
    mc=min_cost(mm)
    mc.export_as_lp(basename=Costname,path='E:\Master2\Buildings')
    mf=min_co2(mm)
    mf.export_as_lp(basename=CO2name,path='E:\Master2\Buildings')
    

Model: BuildingHub
 - number of variables: 306665
   - binary=17532, integer=13, continuous=289120
 - number of constraints: 420549
   - linear=420549
 - parameters: defaults
Model: BuildingHub
 - number of variables: 306665
   - binary=17532, integer=13, continuous=289120
 - number of constraints: 420549
   - linear=420549
 - parameters: defaults
Model: BuildingHub
 - number of variables: 306665
   - binary=17532, integer=13, continuous=289120
 - number of constraints: 420549
   - linear=420549
 - parameters: defaults
Model: BuildingHub
 - number of variables: 306665
   - binary=17532, integer=13, continuous=289120
 - number of constraints: 420549
   - linear=420549
 - parameters: defaults
Model: BuildingHub
 - number of variables: 306665
   - binary=17532, integer=13, continuous=289120
 - number of constraints: 420549
   - linear=420549
 - parameters: defaults
Model: BuildingHub
 - number of variables: 306665
   - binary=17532, integer=13, continuous=289120
 - number of constraints: 

DOcplexException: Unsupported operation: StorCharge_1_4096 + 2.316476432